In [ ]:
using Pkg
Pkg.activate(@__DIR__)

To do:

* Environment data should have its own axis, not shared with contacts
* Integer variables for contact pairs
* Initial condition modification
* Test that CoPs are in region
* Test that friction cone constraints are satisfied
* Move utilities to appropriate places
* Make foot locations variables
* Fix CoPs during each piece?

In [ ]:
using CentroidalTrajOpt
using LinearAlgebra
using SCIP

In [ ]:
# Initial conditions
c0 = SVector(0.05, 0.1, 1.0)
ċ0 = SVector(0.25, 0.15, 0.1)
g = SVector(0.0, 0.0, -9.81);

In [ ]:
problem = CentroidalTrajectoryProblem(with_optimizer(SCIP.Optimizer, limits_gap=0.05), c0=c0, ċ0=ċ0, g=g);

In [ ]:
result = solve!(problem);

In [ ]:
using Test
c = result.center_of_mass
ċ = map_subfunctions(x -> map_elements(derivative, x), c)
c̈ = map_subfunctions(x -> map_elements(derivative, x), ċ)
rs = result.centers_of_pressure
fs = result.contact_forces
τns = result.contact_normal_torques
ns = normals(problem)

# Times
@test issorted(result.break_times)
@test all(x -> x >= 0, result.break_times)

T = last(result.break_times)

# Initial and final conditions
@test c(0) ≈ c0 atol=1e-12
@test ċ(0) ≈ ċ0 atol=1e-12
@test ċ(T) ≈ zeros(3) atol=1e-12
@test c̈(T) ≈ zeros(3) atol=1e-12

for t in range(0, T, length=100)
    ftot = sum(fs[contact](t) for contact in problem.contacts.val)
    
    # Dynamics
    @test c̈(t) ≈ g + ftot atol=1e-5
    
    # Torque about CoM
    τ = sum((rs[j](t) - c(t)) × fs[j](t) + ns[j] * τns[j](t) for j in problem.contacts.val)
    @test τ ≈ zeros(3) atol=1e-4
    
    # Friction cones
    for j in problem.contacts.val
        f = fs[j]
        n = ns[j]
        τn = τns[j]
        
        f_t = f(t)
        fn_t = f_t ⋅ n
        τn_t = τn(t)
        @test fn_t >= 0
        
        # TODO: μ checks
#         @test norm(f_t - n * fn_t) <= μ * fn_t + 1e-6
#         @test -μrot * fn_t <= τn_t
#         @test τn_t <= μrot * fn_s
    end
end

# Continuity around breaks
for t in result.break_times
    @test c(t - 1e-8) ≈ c(t + 1e-8) atol=1e-5
    @test ċ(t - 1e-8) ≈ ċ(t + 1e-8) atol=1e-5
end